<a href="https://colab.research.google.com/github/canamac/zeka-can/blob/main/100_coin_excel_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
import time
import numpy as np
from datetime import datetime

# API Ayarları
COINGECKO_URL = "https://api.coingecko.com/api/v3/coins/markets"
HEADERS = {'User-Agent': 'Mozilla/5.0'}

# 100 Coin Verisini Çek
def get_top_100_coins():
    try:
        params = {
            'vs_currency': 'usd',
            'order': 'market_cap_desc',
            'per_page': 100,
            'page': 1,
            'sparkline': False
        }
        response = requests.get(COINGECKO_URL, headers=HEADERS, params=params, timeout=15)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        print(f"Hata: {str(e)}")
        return None

# Veriyi İşle
def process_data(data):
    if not data:
        return pd.DataFrame()

    df = pd.DataFrame(data)
    df = df[[
        'symbol', 'current_price', 'high_24h', 'low_24h',
        'price_change_percentage_24h', 'market_cap', 'total_volume'
    ]]

    df.rename(columns={
        'symbol': 'Sembol',
        'current_price': 'Fiyat ($)',
        'high_24h': '24s En Yüksek',
        'low_24h': '24s En Düşük',
        'price_change_percentage_24h': 'Değişim (%)',
        'market_cap': 'Piyasa Değeri',
        'total_volume': 'Hacim (24s)'
    }, inplace=True)

    return df

# Sinyal Üret
def generate_signals(df):
    if df.empty:
        return df

    # AL sinyali için koşullar
    df['Sinyal'] = np.where(
        (df['Değişim (%)'] > 5) & (df['Hacim (24s)'] > 1e6),
        'AL',
        'BEKLE'
    )
    return df

# Excel'e Kaydet
def save_to_excel(df):
    try:
        file_name = "crypto_signals.xlsx"  # Sabit dosya adı
        df.to_excel(file_name, index=False, engine='openpyxl')
        print(f"Veriler '{file_name}' dosyasına kaydedildi.")
        return True
    except Exception as e:
        print(f"Excel kaydetme hatası: {str(e)}")
        return False

# Ana Fonksiyon
def main():
    while True:
        print(f"\nTarama Başlatıldı: {datetime.now().strftime('%H:%M:%S')}")
        data = get_top_100_coins()
        if data:
            df = process_data(data)
            df = generate_signals(df)
            if not df.empty:
                save_to_excel(df)
        time.sleep(300)  # 5 dakikada bir

if __name__ == "__main__":
    main()


Tarama Başlatıldı: 18:57:54
Veriler 'crypto_signals.xlsx' dosyasına kaydedildi.

Tarama Başlatıldı: 19:02:55
Veriler 'crypto_signals.xlsx' dosyasına kaydedildi.
